In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
from eth_defi.uniswap_v2.pair import PairDetails, fetch_pair_details

In [3]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 1000

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = fetch_pair_details(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Sync at block:60,592,918 tx:0x4e439b6f156d47c16ede8b36d054b994c1ca0c0457d5d9e0d45be366eb72fc4b
Sync at block:60,592,918 tx:0x267e4da04b94a5d8b28135ba85b831b355ead3a67d2646daa502f7b6032a8ee7
Sync at block:60,592,920 tx:0x9d9231f2be84d0682d108c5f1eaf45ff376b1010961317c05f7096e3c6c0df5e
Sync at block:60,592,921 tx:0x4422b8864a9189459b8124f8d45152f7db13d9f941e5986600739e9a8a1b5e2d
Sync at block:60,592,921 tx:0x4f1503bd2c0b2db942676cd9d6fcad583f5112f521b5c5f5c77bf1db3b58f33c
Sync at block:60,592,923 tx:0xe3538f465ad823b768b2b232c5cab53b86e3e8cce552081332b900b703bebb67
Sync at block:60,592,923 tx:0xe3538f465ad823b768b2b232c5cab53b86e3e8cce552081332b900b703bebb67
Sync at block:60,592,928 tx:0xbb9ce24bc0e40a5a8020fbd0324fcf458ffd7be85d685a53c30671c32cdb2ddb
Sync at block:60,592,928 tx:0x0cbb993ef9746be6998637260318d34e4fc4e16bb2381cfd1e11f27832615acf
Sync at block:60,592,929 tx:0xe9a624bb42ed0eca78617ca63e96b0891d3b58eff4338073683d4dea41cb7e11
Sync at block:60,592,929 tx:0x936a1cf76ad558ef9265

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,sync,sushi,0x348b9b0f279678767155fe52ecbd9e2013bc0748,0x4e439b6f156d47c16ede8b36d054b994c1ca0c0457d5...,60592918,1723667745,{'web3_type': <class 'web3._utils.datatypes.Sy...
1,polygon,uniswapv2pair,sync,sushi,0xff2bbcb399ad50bbd06debadd47d290933ae1038,0x267e4da04b94a5d8b28135ba85b831b355ead3a67d26...,60592918,1723667745,{'web3_type': <class 'web3._utils.datatypes.Sy...
2,polygon,uniswapv2pair,sync,sushi,0x187961fa50d0380d1d42fd0b5e8eda0441d27479,0x9d9231f2be84d0682d108c5f1eaf45ff376b10109613...,60592920,1723667749,{'web3_type': <class 'web3._utils.datatypes.Sy...
3,polygon,uniswapv2pair,sync,sushi,0x5ce7c6533c6a74842386ba744b1278640460d6f8,0x4422b8864a9189459b8124f8d45152f7db13d9f941e5...,60592921,1723667751,{'web3_type': <class 'web3._utils.datatypes.Sy...
4,polygon,uniswapv2pair,sync,sushi,0x5786b267d35f9d011c4750e0b0ba584e1fdbead1,0x4f1503bd2c0b2db942676cd9d6fcad583f5112f521b5...,60592921,1723667751,{'web3_type': <class 'web3._utils.datatypes.Sy...
...,...,...,...,...,...,...,...,...,...
1083,polygon,uniswapv2pair,sync,sushi,0xb0dc320ea9eea823a150763abb4a7ba8286cd08b,0x6bf62d249f7700024c574ace30764efdfbed0f1f4cd0...,60593910,1723669853,{'web3_type': <class 'web3._utils.datatypes.Sy...
1084,polygon,uniswapv2pair,sync,sushi,0x5a70ecedcb737b49a9b26560d0d77394924fef0d,0xa4b611c6b58e12db9513bd33ec4ee5ae1c2ce28ff47c...,60593911,1723669855,{'web3_type': <class 'web3._utils.datatypes.Sy...
1085,polygon,uniswapv2pair,sync,sushi,0x032d86f50ee9c0c361f268bd7c08013465e29968,0xb0a3e15f327f994f0c13a47aba2cb5e9ff88787bd948...,60593913,1723669859,{'web3_type': <class 'web3._utils.datatypes.Sy...
1086,polygon,uniswapv2pair,sync,sushi,0x5a70ecedcb737b49a9b26560d0d77394924fef0d,0x7af7e505a4fba00a05d71a2e096e842e64b3e4da551b...,60593913,1723669859,{'web3_type': <class 'web3._utils.datatypes.Sy...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x348b9b0f279678767155fe52ecbd9e2013bc0748',
  'tx_hash': '0x4e439b6f156d47c16ede8b36d054b994c1ca0c0457d5d9e0d45be366eb72fc4b',
  'blk_num': 60592918,
  'timestamp': 1723667745,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
   'token1': '0x9627a3d6872bE48410fCEce9b1dDD344Bf08c53e',
   'token0_symbol': 'WMATIC',
   'token1_symbol': 'ACE',
   'token0_decimal': 18,
   'token1_decimal': 2,
   'amount0': 1557233524083657204921,
   'amount1': 341071638,
   'price': 2190.2407874291116}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0xff2bbcb399ad50bbd06debadd47d290933ae1038',
  'tx_hash': '0x267e4da04b94a5d8b28135ba85b831b355ead3a67d2646daa502f7b6032a8ee7',
  'blk_num': 60592918,
  'timestamp': 1723667745,
  'details': {'web3_type': web3._utils.datatyp